In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from sklearn.linear_model import SGDClassifier
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
%%writefile basiccount.py
import operator
import numpy as np
from typing import List
from collections import namedtuple
import traceback
import sys


HistMatchResult = namedtuple("HistMatchResult", "idx length")


def find_all_longest(seq, max_len=None) -> List[HistMatchResult]:
    """
    Find all indices where end of `seq` matches some past.
    """
    result = []

    i_search_start = len(seq) - 2

    while i_search_start > 0:
        i_sub = -1
        i_search = i_search_start
        length = 0

        while i_search >= 0 and seq[i_sub] == seq[i_search]:
            length += 1
            i_sub -= 1
            i_search -= 1

            if max_len is not None and length > max_len:
                break

        if length > 0:
            result.append(HistMatchResult(i_search_start + 1, length))

        i_search_start -= 1

    result = sorted(result, key=operator.attrgetter("length"), reverse=True)

    return result


class Pred:
    def __init__(self, *, alpha):
        self.offset_probs = np.full(fill_value=1/3, shape=3)
        self.alpha = alpha
        self.last_feat = None

    def train(self, target):
        if self.last_feat is not None:
            offset = (target - self.last_feat) % 3

            self.offset_probs *= (1 - self.alpha)  # technically you should better decay towards the mean instead?
            self.offset_probs[offset] += self.alpha

    def predict(self, feat):
        result = np.roll(self.offset_probs, shift=feat)
        self.last_feat = feat
        return result
    
    
class BaseAgent:
    def __init__(self):
        self.my_hist = []
        self.opp_hist = []
        self.my_opp_hist = []
        self.outcome_hist = []
        self.step = None

    def __call__(self, obs, conf):
        try:
            if obs.step == 0:
                action = np.random.choice(3)
                self.my_hist.append(action)
                return action

            self.step = obs.step

            opp = int(obs.lastOpponentAction)
            my = self.my_hist[-1]

            self.my_opp_hist.append((my, opp))
            self.opp_hist.append(opp)

            outcome = {0: 0, 1: 1, 2: -1}[(my - opp) % 3]
            self.outcome_hist.append(outcome)

            action = self.action()

            self.my_hist.append(action)

            return action
        except Exception:
            traceback.print_exc(file=sys.stderr)
            raise

    def action(self):
        """
        This has to be implemented to return the action
        """
        pass


class Agent(BaseAgent):
    def __init__(self, alpha=0.01):
        super().__init__()
        self.predictor = Pred(alpha=alpha)

    def action(self):
        self.train()
        probs = self.preds()
        return np.random.choice(3, p=probs)

    def train(self):
        last_beat_opp = (self.opp_hist[-1] + 1) % 3
        self.predictor.train(last_beat_opp)

    def preds(self):
        hist_match = find_all_longest(self.my_opp_hist, max_len=20)

        if not hist_match:
             return [1/3,1/3,1/3]

        feat = self.opp_hist[hist_match[0].idx]
        pred = self.predictor.predict(feat)
        return pred
    
    
agent = Agent()


def call_agent(obs, conf):
    return agent(obs, conf)

In [ ]:
from kaggle_environments import evaluate, make, utils

env = make("rps")
env.reset()
env.run(["basiccount.py", "statistical"]);
env.render(mode="ipython", width=600, height=450)

In [ ]:
def helper(opp_moves):
    features=[]
    map_moves = {0: (1, 0, 0), 1: (0, 1, 0), 2: (0, 0, 1)}
    for move in opponent_moves:
        features.extend(map_moves.get(move))
    return features

In [ ]:
class MyAgent(object):

    def __init__(self):
        self.cls = SGDClassifier(warm_start=True)

    def train(self, opponent_moves):
        x, y = opponent_moves[0:-1], [opponent_moves[-1]]
        new_features = build_features(x)
        x = np.array(new_features).reshape((1, -1))
        self.cls.partial_fit(x, y, classes=[0, 1, 2])

    def opponent_next_move(self, opponent_moves):
        x = opponent_moves[1:]
        new_features = build_features(x)
        x = np.array(new_features).reshape((1, -1))
        return self.cls.predict(x)[0]

In [ ]:
%%writefile prediction.py
def my_agent(observation, configuration):

    if observation.step > 0:

        # This if statement will ensure that the opponent's 
        # number of moves will not exceed the number of moves
        # that can be stored, erasing the oldest move.
        if len(my_agent.opponent_moves) >= my_agent.len_opponent_moves:
            del my_agent.opponent_moves[0]

        my_agent.opponent_moves.append(observation.lastOpponentAction) # Append the last opponent play
        my_agent.ai.train(my_agent.opponent_moves) # Training the agent using the last turn
        opponent_next_move = my_agent.ai.opponent_next_move(my_agent.opponent_moves) # Predicting the next move
    
        # Once we have the most likely opponet's play
        # we can check which of the options (rock, paper, scissors)
        # will ensure our victory
        for i in (0, 1, 2):
            ans = winner(i, opponent_next_move)
            if ans == (1, 0):
                return i
        return observation.lastOpponentAction
    else:
        my_agent.len_opponent_moves = 51 # Number opponent moves storage
        my_agent.opponent_moves = my_agent.len_opponent_moves * [0] # List of last nth opponent moves
        my_agent.ai = MyAgent() # Agent object for training and preditions
        return 0

